In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', encoding = 'windows-1251', sep = ';')

In [3]:
df.shape

(119390, 21)

In [4]:
df.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

In [5]:
df.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [6]:
#Список названий колонок до преобразования
df.columns

Index(['Hotel', 'Is Canceled', 'Lead Time', 'arrival full date',
       'Arrival Date Year', 'Arrival Date Month', 'Arrival Date Week Number',
       'Arrival Date Day of Month', 'Stays in Weekend nights',
       'Stays in week nights', 'stays total nights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'Reserved Room Type', 'Assigned room type',
       'customer type', 'Reservation Status', 'Reservation status_date'],
      dtype='object')

In [7]:
#Поскольку вручную оформлять большой словарь из старых и новых названий долго и высока вероятность ошибки,
#я решил попробовать решить эту задачу через цикл.
for name_column in df.columns:
    #Получение нового названия столбца. replace заменяет пробел на нижнее подчеркивание, ловер переводит строку в нижний регист
    new_name_column = name_column.replace(' ', '_').lower()
    # Замена на редактированное название столбца
    df.rename(columns={name_column:new_name_column}, inplace=True)
#Цикл изначально не изменял df. Я не знал, что для замены нужно указать inplace = True.

In [8]:
#Список названий колонок после преобразования
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [9]:
# Задача №1: Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
success_country_reserved = df\
.query('reservation_status == "Check-Out"')\
.groupby('country', as_index = False)\
.aggregate({'arrival_full_date':'count'})\
.sort_values('arrival_full_date', ascending = False)\
.rename(columns=({'arrival_full_date': 'count_visits'}))

#Произвел проверку по reservation_status, из логики, что если бронирование было успешным, то человек прошел регистрацию, после сделал отметку об уходе.

In [10]:
success_country_reserved.head(5)

,country,count_visits
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [11]:
# Поскольку в df был специальный столбец is_canceled, отвечающий за успешность регистрации
# Я решил пересчитать по нему, и проверить совершил ли я ошибку не используя его.
success_country_reserved_2 = df\
.query('is_canceled == 0')\
.groupby('country', as_index = False)\
.aggregate({'arrival_full_date':'count'})\
.sort_values('arrival_full_date', ascending = False)\
.rename(columns=({'arrival_full_date': 'count_visits'}))

In [12]:
#Ошибки нет.
success_country_reserved_2.head() 

,country,count_visits
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


In [13]:
# Задача №2: На сколько ночей в среднем бронируют отели разных типов?
average_nights = df.groupby('hotel').stays_total_nights.mean().round(2) # Группировака по типу отеля; среднее общего количества ночей одной брони
# Округление до двух знаков после запятой
average_nights

hotel
City Hotel      2.98
Resort Hotel    4.32
Name: stays_total_nights, dtype: float64

In [14]:
# Задача: Найти количесво случаев когда забронированный тип номера отличается от типа номер для заселения
df\
.query('reserved_room_type != assigned_room_type')\
.aggregate({'reserved_room_type' : 'count'})

reserved_room_type    14917
dtype: int64

In [15]:
# Задача №3: На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?
total_month_visit_2016 = df\
.query('arrival_date_year == 2016')\
.groupby('arrival_date_month', as_index=False)\
.aggregate({'is_canceled':'count'})\
.sort_values('is_canceled', ascending = False)\
.rename(columns={'is_canceled' : 'count_visit'})
total_month_visit_2016.head(1) # В 2016 году самым популярным месяцем стал октябрь

,arrival_date_month,count_visit
10,October,6203


In [16]:
total_month_visit_2017 = df\
.query('arrival_date_year == 2017')\
.groupby('arrival_date_month', as_index=False)\
.aggregate({'is_canceled':'count'})\
.sort_values('is_canceled', ascending = False)\
.rename(columns={'is_canceled' : 'count_visit'})
total_month_visit_2017.head(1) # В 2017 году самым популярным месяцем стал Май.

,arrival_date_month,count_visit
7,May,6313


In [50]:
# Задача №4: Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа 
# City Hotel отменялись чаще всего в каждый из периодов.
df\
.query('hotel == "City Hotel" and is_canceled == 1')\
.groupby(['arrival_date_year', 'arrival_date_month'], as_index=False)\
.aggregate({'reservation_status':'count'})\
.sort_values('reservation_status', ascending = False)
#В своем решении я в ручную нашел месяца с наибольшим количеством отмен по годам

,arrival_date_year,arrival_date_month,reservation_status
25,2017,May,2217
16,2016,October,1947
18,2017,April,1926
23,2017,June,1808
12,2016,June,1720
17,2016,September,1567
5,2015,September,1543
6,2016,April,1539
14,2016,May,1436
15,2016,November,1360


In [51]:
# На курсе предложили альтернативное решение.
# Вместо count использовали sum значений is_canceled поскольку там записаны 1/0.
df.query('hotel == "City Hotel"').groupby(['arrival_date_year', 'arrival_date_month']).is_canceled.sum().groupby('arrival_date_year').nlargest(1)
#Чтобы не искать в таблице самостоятельно ответ, был применен метод nlargest, которые выводит строки по убыванию в каждой группе.
#Для этого была произведена дополнительна группировка по году. Визуально, дублирование группировки выглядит неказисто, но как уйти от этого я не нашел.

arrival_date_year  arrival_date_year  arrival_date_month
2015               2015               September             1543
2016               2016               October               1947
2017               2017               May                   2217
Name: is_canceled, dtype: int64

In [62]:
# Задача №5: какая из трёх колонок: adults, children и babies имеет наибольшее среднее значение?
df[['adults', 'children','babies']].mean()
#Из df взято 3 колонки и к каждой из них применен метод mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [65]:
# Задача №6: Создайте колонку total_kids, объединив столбцы children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?
df['total_kids'] = df['children'] + df['babies']
df.groupby('hotel').total_kids.mean().round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

In [76]:
# Задача №7: Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе.
df['has_kids'] = df['total_kids'] > 0
# Чтобы получить в таблице True или false можно просто передать условие и всё 0_0 оч удобно

In [101]:
# Расчет churn_rate для посетителей с детьми
with_children = len(df.query('has_kids == True'))
in_canceled_with_children = len(df.query('has_kids == True and is_canceled == 1'))
print('Общее число бронирований с детьми:', with_children)
print('Число отмен бронирования с детьми:', in_canceled_with_children)
churn_rate_with_children = round(in_canceled_with_children/with_children, 4) * 100
print('Процент отказов посетителей с детьми:', churn_rate_with_children,'%')

Общее число бронирований с детьми: 9332
Число отмен бронирования с детьми: 3259
Процент отказов посетителей с детьми: 34.92 %


In [100]:
# Расчет churn_rate для посетителей без детей
without_children = len(df.query('has_kids == False'))
in_canceled_without_children = len(df.query('has_kids == False and is_canceled == 1'))
print('Общее число бронирований без детей:', without_children)
print('Число отмен бронирования без детьми:', in_canceled_without_children)
churn_rate_without_children = round(in_canceled_without_children/without_children, 4) * 100
print('Процент отказов посетителей без детьми:', churn_rate_without_children,'%')
# Более правильно округлять после умножения на 100, тк я потерял из-за этого сотые доли в первый раз.

Общее число бронирований без детей: 110058
Число отмен бронирования без детьми: 40965
Процент отказов посетителей без детьми: 37.22 %
